In [59]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [60]:
df = pd.read_csv('financial_report_data.csv', dtype={'tickers': str})

In [61]:
urls = {
    'notice_of_shareholders_meeting': "https://money.finance.sina.com.cn/corp/go.php/vGP_StockHolderMeeting/stockid/{ticker}.phtml",
    'litigation_and_arbitration': "https://money.finance.sina.com.cn/corp/go.php/vGP_Lawsuit/stockid/{ticker}.phtml",
    'external_guarantee': "https://money.finance.sina.com.cn/corp/go.php/vGP_Assurance/stockid/{ticker}.phtml"
}

# define functions to crawl data from a single link
def fetch_data(url, ticker, col_name):
    response = requests.get(url.format(ticker=ticker))
    soup = BeautifulSoup(response.text, 'lxml')
    times = 0
    td_elements = soup.find_all('td', text="公告日期")

    if td_elements:
        # find every “announcement date”
        for i, td_element in enumerate(td_elements, start=1):
            # find the first <td>
            next_td = td_element.find_next('td')
            times += 1
            # if next_td:
            #     print(f"第 {i} 个 '公告日期' 后面的 <td> 内容:", next_td
            # else:
            #     print(f"未找到第 {i} 个 '公告日期' 后面的 <td>")
    # else:
    #     print("未找到包含 '公告日期' 的 <td>")




    return col_name, times

# define functions to use multi-threading to crawl multiple links
def fetch_all_data(ticker):
    results = {}
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = []
        for col_name, url in urls.items():
            futures.append(executor.submit(fetch_data, url, ticker, col_name))
        for future in futures:
            col_name, data = future.result()
            results[col_name] = data
    return results

# iterate every ticker and scrape data
for index, row in df.iterrows():
    ticker = row['tickers']
    data = fetch_all_data(ticker)
    for col, value in data.items():
        df.at[index, col] = value


/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_63681/2809751989.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td_elements = soup.find_all('td', text="公告日期")
/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_63681/2809751989.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td_elements = soup.find_all('td', text="公告日期")
encoding error : input conversion failed due to input error, bytes 0x85 0xAC 0xE5 0x91
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x97 0xA5 0xE6 0x9C
encoding error : input conversion failed due to input error, bytes 0x8F 0x90 0xE4 0xBE
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x90 0xE4 0xBE 0x9B
encoding error : input conversion failed due to input error, bytes 0x87 0x91 0xE9 0xA2
I/O error : encoder error
encoding error : input 

ConnectionError: HTTPSConnectionPool(host='money.finance.sina.com.cn', port=443): Max retries exceeded with url: /corp/go.php/vGP_Lawsuit/stockid/000597.phtml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x165f93a30>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [98]:
# Continue scraping after connection error
start_ticker = '688598'

# find start ticker in the error above
start_index = df[df['tickers'] == start_ticker].index[0]

for index, row in df.loc[start_index:].iterrows():
    ticker = row['tickers']
    
    data = fetch_all_data(ticker)
    for col, value in data.items():
        df.at[index, col] = value


/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_63681/2809751989.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td_elements = soup.find_all('td', text="公告日期")
encoding error : input conversion failed due to input error, bytes 0x8B 0x85 0xE4 0xBF
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x8B 0xE6 0x97 0xA5


In [99]:
df.to_csv('company_major_matters.csv', index=False)

In [101]:
missing_values_count = df.isnull().sum()
print(missing_values_count)

tickers                                 0
company                                 0
listed_market                           0
listing_date                            0
issue_price                             1
company_website                         0
business_address                        0
main_business                           0
diluted_earnings_per_share 06/30/24     0
diluted_earnings_per_share 03/31/24     0
diluted_earnings_per_share 12/31/23     0
diluted_earnings_per_share 09/30/23     0
weighted_earnings_per_share 06/30/24    0
weighted_earnings_per_share 03/31/24    0
weighted_earnings_per_share 12/31/23    0
weighted_earnings_per_share 09/30/23    0
net_asset_per_share 06/30/24            0
net_asset_per_share 03/31/24            0
net_asset_per_share 12/31/23            0
net_asset_per_share 09/30/23            0
return_on_equity 06/30/24               0
return_on_equity 03/31/24               0
return_on_equity 12/31/23               0
return_on_equity 09/30/23         